In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('drive/MyDrive/식물을 부탁해/')

Mounted at /content/drive


In [ ]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.4 MB/s eta 0:00:00


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
Setup complete ✅ (12 CPUs, 167.1 GB RAM, 38.3/235.7 GB disk)


In [ ]:
train_path = "/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location_final/train/images"
val_path = "/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location_final/validation/images"

In [ ]:
yaml_content = f"""
train: {train_path}
val: {val_path}

# 클래스 정보
nc: 1
names: ['plant']
"""
yaml_save_path = "/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location_final/data_v2.yaml"

with open(yaml_save_path, 'w') as f:
    f.write(yaml_content)

print(f"✅ YAML 파일 생성 완료: {yaml_save_path}")

✅ YAML 파일 생성 완료: /content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location_final/data_v2.yaml


In [ ]:
# 1. data.yaml 파일 경로
data_yaml_path = "/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location_final/data_v2.yaml"

# 2. YOLOv8 학습(미세조정) 명령어 실행
!yolo train \
  model=yolov8n.pt \
  data="{data_yaml_path}" \
  epochs=150 \
  imgsz=640 \
  batch=16 \
  project="/content/drive/MyDrive/식물을 부탁해/seungwoo/yolo_runs" \
  name="yolo_v2"

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location_final/data_v2.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_v22, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

In [ ]:
import os
import cv2 # OpenCV for image reading/cropping
from ultralytics import YOLO
from google.colab import drive
from tqdm.notebook import tqdm # 진행 상황을 보여주는 바

drive.mount('/content/drive')


# 1. 방금 학습시킨 'best.pt' 모델 파일의 전체 경로
MODEL_PATH = "/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location/runs/plant_detector_v1/weights/best.pt"

# 2. Crop할 '원본' 데이터셋의 최상위 폴더
# (안에 train, validation 폴더가 들어있는 곳)
SOURCE_DATASET_DIR = "/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_dataset_for_yolo" # (★ 사용자님이 알려주신 경로)

# 3. Crop된 이미지들을 '새로' 저장할 최상위 폴더
# (이 폴더는 없으면 코드가 새로 만듭니다)
SAVE_DATASET_DIR = "/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_dataset_cropped_for_mobilenet" # (★ 저장될 새 폴더 이름 예시)

# --- 3. 모델 로드 ---
try:
    model = YOLO(MODEL_PATH)
    print(f"모델 로드 성공: {MODEL_PATH}")
except Exception as e:
    print(f"모델 로드 오류: {e}")

# --- 4. 추론 및 Crop 실행 (폴더 구조 복제) ---
splits = ['train', 'validation'] # 처리할 하위 폴더들

for split in splits:
    source_split_dir = os.path.join(SOURCE_DATASET_DIR, split)
    save_split_dir = os.path.join(SAVE_DATASET_DIR, split)

    # train/validation 폴더가 있는지 확인
    if not os.path.isdir(source_split_dir):
        continue

    # 15개의 식물 종 폴더 목록 가져오기
    species_folders = [f for f in os.listdir(source_split_dir) if os.path.isdir(os.path.join(source_split_dir, f))]
    print(f"--- '{split}' 폴더 처리 시작 ---")

    for species in species_folders:
        source_species_dir = os.path.join(source_split_dir, species)
        save_species_dir = os.path.join(save_split_dir, species)

        # 1. (가장 중요) Crop된 이미지를 저장할 새 폴더 생성
        os.makedirs(save_species_dir, exist_ok=True)

        # 2. 원본 이미지 파일 목록 가져오기
        image_files = [f for f in os.listdir(source_species_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        print(f"'{split}/{species}' 폴더 처리 중... (이미지 {len(image_files)}개)")

        # 3. 개별 이미지 처리
        for filename in tqdm(image_files, desc=f"{split}/{species}"):
            image_path = os.path.join(source_species_dir, filename)

            try:
                # 4. 원본 이미지 읽기
                img = cv2.imread(image_path)
                if img is None:
                    print(f"\n경고: {filename} 읽기 실패.")
                    continue

                # 5. YOLO 추론
                results = model.predict(image_path, verbose=False)
                result = results[0]

                # 6. 탐지된 박스가 있는지 확인
                if len(result.boxes) > 0:
                    # 가장 신뢰도가 높은 박스 1개만 사용
                    box = result.boxes.xyxy[0].cpu().numpy()
                    xmin, ymin, xmax, ymax = int(box[0]), int(box[1]), int(box[2]), int(box[3])

                    # 7. 이미지 Crop
                    cropped_img = img[ymin:ymax, xmin:xmax]

                    # 8. Crop된 이미지 저장
                    save_path = os.path.join(save_species_dir, filename)
                    cv2.imwrite(save_path, cropped_img)
                else:
                    # 모델이 식물을 탐지하지 못한 경우
                    print(f"\n경고: {filename}에서 식물 탐지 실패.")

            except Exception as e:
                print(f"\n오류: {filename} 처리 중 예외 발생 - {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
모델 로드 성공: /content/drive/MyDrive/식물을 부탁해/seungwoo/plant_location/runs/plant_detector_v1/weights/best.pt
--- 'train' 폴더 처리 시작 ---
'train/관음죽' 폴더 처리 중... (이미지 400개)


train/관음죽:   0%|          | 0/400 [00:00<?, ?it/s]


경고: N50-B-2-13-B-1-V-230908-000259.jpg에서 식물 탐지 실패.

경고: N50-B-2-13-L-2-V-230914-001431.jpg에서 식물 탐지 실패.

경고: N50-B-2-13-B-2-V-230826-000490.jpg에서 식물 탐지 실패.

경고: N50-B-2-13-L-3-V-230916-002233.jpg에서 식물 탐지 실패.
'train/금전수' 폴더 처리 중... (이미지 400개)


train/금전수:   0%|          | 0/400 [00:00<?, ?it/s]

'train/디펜바키아' 폴더 처리 중... (이미지 400개)


...

'validation/호접란' 폴더 처리 중... (이미지 100개)


validation/호접란:   0%|          | 0/100 [00:00<?, ?it/s]

'validation/홍콩야자' 폴더 처리 중... (이미지 100개)


validation/홍콩야자:   0%|          | 0/100 [00:00<?, ?it/s]

--- 🌟 모든 작업 완료 🌟 ---
Crop된 새 데이터셋이 '/content/drive/MyDrive/식물을 부탁해/seungwoo/plant_dataset_cropped_for_mobilenet' 폴더에 생성되었습니다.
